# Incremental Unified Framework (IUF) Testing

We want to test each component and module in isolation to make sure it's working properly

_TODO:_
1. Go through ViT code and understand unique implementation
2. Update to include modules for creating Discriminator, Encoder, & Decoder
3. Check for paper/code for architecture or hyperparams match our implementation (num_heads, dim, etc)
4. Latent Space Regularization
5. Gradient Update Regularization


In [1]:
import einops as ein
import torch
import torch.nn as nn
import torch.nn.functional as F
from Methods import BaseAnomalyDetector
from Methods.IUF.ViT import MultiHeadSelfAttention, ViTBlock, ViT
from Methods.IUF.utils.discriminator import Discriminator
from Methods.IUF.utils.encoder import Encoder
from Methods.IUF.utils.decoder import Decoder

In [2]:
# Starting to put together an IUF pipeline for testing
class IUF(BaseAnomalyDetector):
    """
    Complete IUF Module

    We have one ViT class that we build on,
    From there, we will add methods to create the discriminator, encoder, and decoder.

    Algorithm Notes:

        ViT
        - Added positional encoding to the tokens
        - Changed BatchNorm to LayerNorm and ReLU to GELU, in accordance with the ViT paper
        - I noticed that in their Multi-head attention, the authors only work on row-wise attention to simplify their computations, so I am going to operate on patches, as the multiplication is too large.
        - Batch size needs to maybe be larger than the embedding dimension?

        Loss function consists of the following components:
        - Reconstruction error = abs(x_recon - x)
        - Discriminator error = CrossEntropy(discrim_output, true label)
        - Singular Value error = sum(singular_vals from t -> C), from SVD(M_hat),
                                where t is a hyperparameter and C is the total number of singular values.
        Each component is weighted by a corresponding lambda, where
        - lamba1 = 1
        - lamba2 = 0.5
        - lamba3 = 1-10
    """
    def __init__(self):
        super().__init__()

        self.discriminator = Discriminator()

        self.encoder = Encoder()

        self.decoder = Decoder()

        return

    def forward(self, x):

        # oasa_features = list of length num_layers,
        # where each item is a tensor of size (B x L x E)

        # d_out is of size (B x num_classes)
        oasa_features, d_out = self.discriminator(x, return_features=True)

        z, u, s, v = self.encoder(x, oasa_features)
        # z = latent features, (B x L x E)
        # u, s, v from SVD
        # u, (B x B)
        # S, (B/C), whichever is smaller
        # V, (C x C), C = channels/embed_dim

        x_recon = self.decoder(z)

        return x_recon

iuf = IUF()
dummy = torch.rand(8, 3, 224, 224)
x_recon = iuf(dummy)

Using cuda device


In [3]:
x_recon.shape

torch.Size([8, 3, 196, 196])

In [17]:
s

tensor([21.6624,  0.4635,  0.4140,  0.2827,  0.2293,  0.2035,  0.1863,  0.1360],
       grad_fn=<LinalgSvdBackward0>)

In [6]:
.049201**2

0.002420738401